# Multivariate Gaussian Distributions: Marginalization and Conditioning

In [9]:
from typing import Tuple
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [10]:
mu_all = np.zeros(2)
cov_one = np.array([[1.0, 0.5], [0.5, 1.0]])
cov_two = np.array([[1.0, 0.0], [0.0, 1.0]])
cov_three = np.array([[1.0, -0.5], [-0.5, 1.0]])

### Multivariate Gaussian PDF Implementation

In [11]:
def multivariate_gaussian_pdf(x_vec: np.ndarray, mu_vec: np.ndarray, covariance_matrix: np.ndarray) -> float:
    """
    Calculates the probability density function of a multivariate Gaussian.

    Args:
        x_vec: Vector at which to evaluate the pdf.
        mu_vec: Mean of distribution.
        covariance_matrix: Covariance matrix of the distribution.

    Returns:
        Probability density function value at location x_vec.
    """

    d = len(x_vec)
    x_centered = x_vec - mu_vec
    cov_det = np.linalg.det(covariance_matrix)
    constant = 1/(pow(2*math.pi, d/2) * pow(cov_det, 0.5))

    return constant * np.exp(-0.5 * np.transpose(x_centered) * np.linalg.inv(covariance_matrix) * x_centered)

In [12]:
test_point = np.array([1.01, 0.22])
my_pdf = multivariate_gaussian_pdf(test_point, mu_all, cov_one)
scipy_pdf = stats.multivariate_normal.pdf(test_point, mu_all, cov_one)
assert np.round(my_pdf - scipy_pdf, 6) == 0, "The two values do not agree. Did you forget the terms outside the exponential?"

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Comparing the time taken by the above function to scipy

In [ ]:
%%timeit
multivariate_gaussian_pdf(test_point, mu_all, cov_one)

In [ ]:
%%timeit
stats.multivariate_normal.pdf(test_point, mu_all, cov_one)

In [ ]:
# We can use numpy's vectorize capability to vectorize our pdf function.
mg_pdf_vectorized = np.vectorize(multivariate_gaussian_pdf, signature='(n), (n), (n,n) -> ()')

# We can also use numpy's meshgrid to get the points to evaluate
x1_points, x2_points = np.meshgrid(np.linspace(-2,2,100), np.linspace(-2,2,100))
x_vec_points = np.stack([x1_points, x2_points], axis=-1)

In [ ]:
def plot_pdf_and_covariance(covariance_matrix:np.ndarray, color='black'):
    # Use the vectorized function to evaluate the pdf.
    pdf_evaluations = mg_pdf_vectorized(x_vec_points, mu_all, covariance_matrix)
    
    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    ax[0].contour(x1_points, x2_points, pdf_evaluations/np.max(pdf_evaluations), levels=[ np.exp(-1),np.exp(-0.5)], 
                  colors=color, linewidths=3)
    ax[0].set_xlabel(r'$x_1$')
    ax[0].set_ylabel(r'$x_2$')
    ax[0].set_title('pdf')
    ax[1].imshow(covariance_matrix, cmap='bwr', vmax=1, vmin=-1)
    ax[1].set_xticks([0,1])
    ax[1].set_xticklabels([r'$x_1$',r'$x_2$'])
    ax[1].set_yticks([0,1])
    ax[1].set_yticklabels([r'$x_1$',r'$x_2$'])
    ax[1].set_title('Covariance Matrix')

    for x_1 in range(2):
        for x_2 in range(2):
            ax[1].text(x_1, x_2, covariance_matrix[x_1, x_2], fontsize=20, ha='center', va='center')
    
    plt.show()

plot_pdf_and_covariance(cov_one, '#1b9e77')
plot_pdf_and_covariance(cov_two, '#d95f02')
plot_pdf_and_covariance(cov_three, '#7570b3')

### Marginal Distribution of Multivariate Gaussian Distribution 

In [ ]:
def marginal_mean_covariance(mu_vec: np.ndarray, covariance_matrix: np.ndarray, marginal_index:int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculates the marginalized mean and covariance matrix of a multivariate Gaussian.

    Args:
        mu_vec: Mean of distribution.
        covariance_matrix: Covariance matrix of the distribution.
        marginal_index: Variable index to marginalize on.

    Returns:
        Mean and covariance of marginalized Gaussian.

    Notes:
        The easiest way to do this is to permute the vector and matrix so that the mariginal_index is the last index.
        Some functions you may find useful beyond the ones you've already used:
        - np.delete
    """

    marginal_mu = np.delete(mu_vec, marginal_index)
    marginal_cov = np.delete(covariance_matrix, )

    return marginal_mu, marginal_cov

In [ ]:
def marginal_mean_covariance(mu_vec: np.ndarray, covariance_matrix: np.ndarray, marginal_index:int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculates the marginalized mean and covariance matrix of a multivariate Gaussian.

    Args:
        mu_vec: Mean of distribution.
        covariance_matrix: Covariance matrix of the distribution.
        marginal_index: Variable index to marginalize on.

    Returns:
        Mean and covariance of marginalized Gaussian.

    Notes:
        The easiest way to do this is to permute the vector and matrix so that the mariginal_index is the last index.
        Some functions you may find useful beyond the ones you've already used:
        - np.delete
    """

    n = mu_vec.shape[0]

    perm = []
    for i in range(n):
        if (i != marginal_index):
            perm.append(i)

    perm.append(marginal_index)

    mu_perm = mu_vec[perm]
    cov_perm = covariance_matrix[perm]

    marginal_mu = mu_perm[:-1]
    marginal_cov = mu_perm[]

    return marginal_mu, marginal_cov

### Marginal Gaussian PDF

In [ ]:
def marginal_gaussian_pdf(x_vec: np.ndarray, mu_vec: np.ndarray, covariance_matrix: np.ndarray,
                             marginal_index:int) -> float:
    """
    Calculates the marginalized probability density function of a multivariate Gaussian.

    Args:
        x_vec: Vector at which to evaluate the pdf.
        mu_vec: Mean of distribution.
        covariance_matrix: Covariance matrix of the distribution.
        marginal_index: Variable index to marginalize on.

    Returns:
        Probability density function value at location x_vec.
    """
    mu_marg, cov_marg = marginal_mean_covariance(mu_vec, covariance_matrix, marginal_index)

    return multivariate_gaussian_pdf(x_vec, mu_marg, cov_marg)

### Conditional Distribution of Multivariate Gaussian Distribution 

In [ ]:
def conditioned_mean_covariance(mu_vec: np.ndarray, covariance_matrix: np.ndarray, condition_index:int, 
                                condition_value:float) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculates the conditioned mean and covariance matrix of a multivariate Gaussian.

    Args:
        mu_vec: Mean of distribution.
        covariance_matrix: Covariance matrix of the distribution.
        conditioned_index: Variable index to condition on.
        conditioned_value: Value of the variable to condition on.

    Returns:
        Mean and covariance of conditioned Gaussian.

    Notes:
        The easiest way to do this is to permute the vector and matrix so that the condition_index is the last index. 
        Some functions you may find useful beyond the ones you've already used:
        - np.delete
        - np.squeeze
    """

### Conditional Gaussian PDF

In [ ]:
def conditioned_gaussian_pdf(x_vec: np.ndarray, mu_vec: np.ndarray, covariance_matrix: np.ndarray,
                             condition_index:int, condition_value:float) -> float:
    """
    Calculates the conditioned probability density function of a multivariate Gaussian.

    Args:
        x_vec: Vector at which to evaluate the pdf.
        mu_vec: Mean of distribution.
        covariance_matrix: Covariance matrix of the distribution.
        conditioned_index: Variable index to condition on.
        conditioned_value: Value of the variable to condition on.

    Returns:
        Probability density function value at location x_vec.
    """
    mu_cond, cov_cond = conditioned_mean_covariance(mu_vec, covariance_matrix, condition_index, condition_value)

    return multivariate_gaussian_pdf(x_vec, mu_cond, cov_cond)

In [ ]:
# Let's test our functions to make sure they give the right values.
# This block will output nothing if the code above is correct.
# Test the marginal
mu_four = np.array([0.2, 0.3, -0.2])
cov_four = np.array([[1.0, 0.5, 0.2],[0.5, 1.0, 0.5], [0.2, 0.5, 1.0]])
my_mean, my_covariance = marginal_mean_covariance(mu_four, cov_four, marginal_index=1)
correct_mean, correct_covariance = (np.array([0.2, -0.2]), np.array([[1.0, 0.2],[0.2, 1.0]]))
np.testing.assert_array_almost_equal(my_mean, correct_mean)
np.testing.assert_array_almost_equal(my_covariance, correct_covariance)

# Test the conditional
my_mean, my_covariance = conditioned_mean_covariance(mu_four, cov_four, condition_index=1, condition_value=1.3)
correct_mean, correct_covariance = (np.array([0.7, 0.3]), np.array([[0.75, -0.05],[-0.05, 0.75]]))
np.testing.assert_array_almost_equal(my_mean, correct_mean)
np.testing.assert_array_almost_equal(my_covariance, correct_covariance)

Let's consider our three-dimensional covariance matrix and see how marginalizing changes the distribution.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.imshow(cov_four, cmap='bwr', vmax=1, vmin=-1)
ax.set_xticks([0,1,2])
ax.set_xticklabels([r'$x_1$',r'$x_2$',r'$x_3$'])
ax.set_yticks([0,1,2])
ax.set_yticklabels([r'$x_1$',r'$x_2$',r'$x_3$'])
ax.set_title('Covariance Matrix')

for x_1 in range(3):
    for x_2 in range(3):
        ax.text(x_1, x_2, cov_four[x_1, x_2], fontsize=20, ha='center', va='center')

fig, ax = plt.subplots(1, 1, figsize=(10,5))
_, cov_marg_x_three = conditioned_mean_covariance(mu_four, cov_four, condition_index=2, condition_value=0.0)
ax.imshow(cov_marg_x_three, cmap='bwr', vmax=1, vmin=-1)
ax.set_xticks([0,1])
ax.set_xticklabels([r'$x_1$',r'$x_2$'])
ax.set_yticks([0,1])
ax.set_yticklabels([r'$x_1$',r'$x_2$'])
ax.set_title(r'Covariance Matrix Marginalized over $x_3$')

for x_1 in range(2):
    for x_2 in range(2):
        ax.text(x_1, x_2, cov_four[x_1, x_2], fontsize=20, ha='center', va='center')

plt.show()